In [1]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import keras
import onnxruntime
from onnxruntime.quantization import (CalibrationDataReader, CalibrationMethod,
                                      QuantFormat, QuantType, quantize_static)

2024-02-06 23:05:07.098505: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 23:05:07.098568: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 23:05:07.100887: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 23:05:07.112731: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-06 23:05:08.529758: W tensorflow/compiler/tf2

In [54]:
import numpy as np

class DR(CalibrationDataReader):
    def __init__(self):
        self.data = np.memmap('../data/dataset/2023_1600.bin', dtype='int64', mode='r').reshape((-1, 12))
        print(self.data.shape)
        idxs = np.random.choice(self.data.shape[0], 1024 * 64, replace=False)
        self.data = self.data[idxs]
        print(self.data.shape)
        self.index = 0

    def get_next(self):
        batch_size = 1024
        batch = self.data[self.index:self.index+batch_size]

        if len(batch) < batch_size:
            return None

        self.index += batch_size

        return { "input_1": batch }

dr = DR()

quantize_static(
    model_input="../models/preprocessed.onnx",
    model_output="../models/quant.onnx",
    calibration_data_reader=dr,
    quant_format=QuantFormat.QDQ,
    activation_type=QuantType.QInt8,
    weight_type=QuantType.QInt8,
)

(8067723, 12)
(65536, 12)


In [55]:
prep_model = onnxruntime.InferenceSession("../models/preprocessed.onnx")
quantized_model = onnxruntime.InferenceSession("../models/quant.onnx")

a = prep_model.run(output_names=["dense_3"], input_feed={"input_1":dr.data})
b = quantized_model.run(output_names=["dense_3"], input_feed={"input_1":dr.data})

print(a[:10])
print(b[:10])
print(np.abs(a[0] - b[0]).max())

[array([[ 0.71758837],
       [-0.33927637],
       [ 0.09160953],
       ...,
       [ 1.6370492 ],
       [-0.44199055],
       [ 0.17647974]], dtype=float32)]
[array([[ 0.74480486],
       [-0.37240243],
       [ 0.12413415],
       ...,
       [ 1.7378781 ],
       [-0.49653658],
       [ 0.        ]], dtype=float32)]
8.060038
